## Import Required Packages

In [1]:
import sys
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

import time
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, Dropout
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

%load_ext tensorboard

from sklearn.utils import shuffle

import numpy.random as rng
np.random.seed(1337)

## Create Data Generators

In [2]:
SIZE = 256
IMG_SIZE = (SIZE, SIZE)
NUM_CHANNELS = 3
BATCH_SIZE = 16

# Define training augmentations
train_datagen = ImageDataGenerator(
    rescale=1./255,
    #zca_whitening=True,
    rotation_range=40,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.5,
    shear_range=0.2,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True,
    vertical_flip=True)

# Define testing augmentations
test_datagen = ImageDataGenerator(rescale=1./255)#,
    #zca_whitening=True)

In [5]:
# Create example augmented images
img = load_img('F:/Work/University/Year 5/ACS6420_Advanced Project/Code/Data/Post Installation/Binary/Training/All Cases/Both/Correct/IMG_1500.jpg')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0
for batch in train_datagen.flow(x, batch_size=1, save_to_dir='F:/Work/University/Year 5/ACS6420_Advanced Project/Code/Data/preview', save_format='jpeg'):
    i+=1
    if i > 9:
        break

In [12]:
train_generator = train_datagen.flow_from_directory(
    'F:/Work/University/Year 5/ACS6420_Advanced Project/Code/Data/Post Installation/Binary/Training/All Cases/Both',
    target_size=IMG_SIZE,
    color_mode='rgb',
    shuffle=True,
    seed=1,
    batch_size=BATCH_SIZE,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    'F:/Work/University/Year 5/ACS6420_Advanced Project/Code/Data/Post Installation/Binary/Validation/All Cases/Both',
    target_size=IMG_SIZE,
    color_mode='rgb',
    shuffle=True,
    seed=2,
    batch_size=BATCH_SIZE,
    class_mode='binary')

Found 140 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


## Create Convolutional Neural Network

In [13]:
base_logdir = "logs/scalars/"
for f in os.listdir(base_logdir):
    file_path = os.path.join(base_logdir, f)
    
    shutil.rmtree(file_path)

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [4]:
# Convolutional Neural Network
model = Sequential()
model.add(Conv2D(16, (3,3), input_shape=(SIZE, SIZE, NUM_CHANNELS), data_format='channels_last', kernel_regularizer=l2(1E-2), bias_regularizer=l2(1E-2)))
model.add(BatchNormalization())
model.add(layers.Activation(activations.relu))
model.add(Dropout(0.3))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), kernel_regularizer=l2(1E-2), bias_regularizer=l2(1E-2)))
model.add(BatchNormalization())
model.add(layers.Activation(activations.relu))
model.add(Dropout(0.3))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), kernel_regularizer=l2(1E-2), bias_regularizer=l2(1E-2)))
model.add(BatchNormalization())
model.add(layers.Activation(activations.relu))
model.add(Dropout(0.3))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(128, kernel_regularizer=l2(1E-2), bias_regularizer=l2(1E-2)))
model.add(BatchNormalization())
model.add(layers.Activation(activations.relu))
model.add(Dropout(0.3))

model.add(Dense(1, activation='sigmoid'))

model.summary()

optimizer = Adam(lr=0.0001)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy', 'Precision', 'Recall'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 254, 254, 16)      448       
_________________________________________________________________
batch_normalization_4 (Batch (None, 254, 254, 16)      64        
_________________________________________________________________
activation_4 (Activation)    (None, 254, 254, 16)      0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 254, 254, 16)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 127, 127, 16)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 125, 125, 32)      4640      
_________________________________________________________________
batch_normalization_5 (Batch (None, 125, 125, 32)     

In [15]:
model.fit(train_generator, steps_per_epoch=train_generator.samples//BATCH_SIZE, 
          validation_data=validation_generator, validation_steps=validation_generator.samples//BATCH_SIZE,
          epochs=25, callbacks=[tensorboard_callback], verbose=1)

Epoch 1/25
1/8 [==>...........................] - ETA: 0s - loss: 4.0586 - accuracy: 0.4375 - precision: 0.2857 - recall: 0.3333WARNING:tensorflow:From f:\work\university\year 5\acs6420_advanced project\code\siamese_env\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
8/8 [==============================] - 18s 2s/step - loss: 3.7777 - accuracy: 0.6452 - precision: 0.6545 - recall: 0.5902 - val_loss: 3.8116 - val_accuracy: 0.5104 - val_precision: 0.5104 - val_recall: 1.0000
Epoch 2/25
8/8 [==============================] - 18s 2s/step - loss: 3.4970 - accuracy: 0.8468 - precision: 0.8909 - recall: 0.7903 - val_loss: 3.7500 - val_accuracy: 0.4323 - val_precision: 0.4556 - val_recall: 0.8817
Epoch 3/25
8/8 [==============================] - 18s 2s/step - loss: 3.4627 - accuracy: 0.8047 - precision: 0.8197 - 

In [18]:
%tensorboard --logdir logs/scalars

Reusing TensorBoard on port 6006 (pid 15796), started 0:06:12 ago. (Use '!kill 15796' to kill it.)